In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from csfloat import *

In [65]:
import pandas as pd

In [4]:
# https://docs.csfloat.com/
# https://github.com/Rushifakami/csfloat_api
# https://github.com/Revadike/InternalSteamWebAPI/wiki
# https://github.com/Revadike/InternalSteamWebAPI/wiki/Get-Market-Price-History

In [66]:
my_orders = load_orders_book()

In [68]:
len(my_orders), my_orders[:2]

(90,
 [{'date': '2025-09-24', 'name': 'Sticker | Lefty (CT)', 'paid': 0.43},
  {'date': '2025-09-24', 'name': 'Sticker | Lefty (CT)', 'paid': 0.43}])

In [67]:
pd.DataFrame(my_orders).head(6)

,date,name,paid
0,2025-09-24,Sticker | Lefty (CT),0.43
1,2025-09-24,Sticker | Lefty (CT),0.43
2,2025-09-24,Sticker | Ribbon Tie,0.43
3,2025-09-24,Sticker | Clown Nose,0.43
4,2025-09-24,Sticker | Lefty (T),0.43
5,2025-09-24,Sticker | Say Cheese (Holo),0.43


In [10]:
current_prices_data = gather_current_prices(my_orders)

100%|██████████████████████████████████████████████████████████████| 23/23 [01:06<00:00,  2.90s/it]


In [12]:
pd.DataFrame(current_prices_data).head(6)

,name,sell_price,estimated_price,buy_price,sell_history_avg_qty,sell_history_avg_price
0,Sticker | Taste Buddy (Holo),7.89,5.76,7.80,397,811.25
1,Sticker | XD,0.13,0.07,0.12,260,10.20
2,Sticker | Strike A Pose,0.09,0.05,0.08,220,8.60
3,Sticker | Quick Peek,0.35,0.22,0.33,342,34.38
4,Sticker | Hypnoteyes (Holo),1.85,1.11,1.83,377,179.58
5,Sticker | Kawaii Eyes (Glitter),1.54,1.13,1.47,254,154.97


In [59]:
import math
CSF_SELL_FEE = 0.021

In [69]:
price_ref = 'buy_price'

results = []
for buy in my_orders:
    current_info = [i for i in current_prices_data if i['name'] == buy['name']][0]
    my_sell_price = math.floor(current_info[price_ref] * 100 * (1 - CSF_SELL_FEE)) / 100
    doc = {
        'paid': buy['paid'],
        'profit': int(my_sell_price > buy['paid']),
        'rev': my_sell_price
    }
    results.append(doc)
print(f'transactions: {len(results)}')
profitable_perc = round(sum([i ["profit"] for i in results]) / len(results) * 100)
print(f'profitable:   {profitable_perc}%')
spent = sum([i["paid"] for i in results])
print(f'total spent:  {spent}')
rev = sum([i["rev"] for i in results])
print(f'total rev:    {rev}')
roi = round(rev / spent * 100)
print(f'roi:          {roi}%')

transactions: 90
profitable:   43%
total spent:  38.7
total rev:    90.48
roi:          234%
